Даны результаты работы трех машинных переводчиков (N,P,Z) на небольших выборках переводов с английского на немецкий, с английского на русский и с казахского на русский. Стандартная оценка качества перевода производится с использованием специальной метрики BLEU.

Требуется определить для каких пар языков можно выделить какого-либо лидера среди переводчиков. Для BLEU учитывать только слова, регистр не учитывать.

# Все должно работать, если в директорию, где лежит ноутбук поместить папку $data$ со всеми $.txt-$файлами

In [7]:
import numpy as np
import os
from nltk.tokenize import RegexpTokenizer

In [8]:
os.getcwd()

'/home/nullkatar/Документы/psad_homework/lab1'

In [18]:
names = sorted(os.listdir(os.getcwd()))
names.remove('.ipynb_checkpoints')
names.remove('README.md')
names.remove('lab1_5.ipynb')
names.remove('lab1_8.ipynb')
names.remove('lab1_3-Copy1.ipynb')
names.remove('laba.ipynb')
names

['en_de_gold.txt',
 'en_de_n.txt',
 'en_de_orig.txt',
 'en_de_p.txt',
 'en_de_z.txt',
 'en_ru_gold.txt',
 'en_ru_n.txt',
 'en_ru_orig.txt',
 'en_ru_p.txt',
 'en_ru_z.txt',
 'kk_ru_gold.txt',
 'kk_ru_n.txt',
 'kk_ru_orig.txt',
 'kk_ru_p.txt',
 'kk_ru_z.txt']

In [19]:
tokenizer = RegexpTokenizer(r'\w+')

In [20]:
tok = []

for name in names[0:5]:
    f = open(name, 'r')
    l = [line.lower().strip() for line in f]
    tok.append([tokenizer.tokenize(g) for g in l])
    f.close()

In [21]:
import nltk.translate.bleu_score as score
import pandas as pd

In [22]:
en_de = np.zeros((10, 3))

for i in range(10):
    k = 0
    for j in (1,3,4):
        #en_de[i,k] = score.modified_precision(tok[j][i], tok[0][i], n = 1)
        en_de[i,k] = score.sentence_bleu(tok[j][i], tok[0][i], smoothing_function = score.SmoothingFunction().method2)
        k += 1

print('English to Deutsche:')
pd.DataFrame(data = en_de, columns = ['N', 'P', 'Z'])

English to Deutsche:


,N,P,Z
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.072239,0.072239,0.068113
4,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000


In [23]:
tok = []

for name in names[5:10]:
    f = open(name, 'r')
    l = [line.lower().strip() for line in f]
    tok.append([tokenizer.tokenize(g) for g in l])
    f.close()

In [24]:
en_ru = np.zeros((10, 3))

for i in range(10):
    k = 0
    for j in (1,3,4):
        #en_ru[i,k] = score.modified_precision(tok[j][i], tok[0][i], n = 1)
        en_ru[i,k] = score.sentence_bleu(tok[j][i], tok[0][i], smoothing_function = score.SmoothingFunction().method1, weights=(1/2, 1/2, 0 , 0))
        k += 1

print('English to Russian:')
pd.DataFrame(data = en_ru, columns = ['N', 'P', 'Z'])

English to Russian:


,N,P,Z
0,0.043861,0.047673,0.043861
1,0.023870,0.023870,0.023870
2,0.030861,0.030861,0.030861
3,0.014712,0.014712,0.014712
4,0.017541,0.017541,0.012403
5,0.027524,0.027524,0.027524
6,0.030151,0.030151,0.027531
7,0.035806,0.035806,0.035806
8,0.021822,0.021822,0.021822
9,0.030861,0.030861,0.037796


In [25]:
tok = []

for name in names[10:]:
    f = open(name, 'r')
    l = [line.lower().strip() for line in f]
    tok.append([tokenizer.tokenize(g) for g in l])
    f.close()

In [26]:
kk_ru = np.zeros((10, 3))

for i in range(10):
    k = 0
    for j in (1,3,4):
        #kk_ru[i,k] = score.modified_precision(tok[j][i], tok[0][i], n = 1)
        kk_ru[i,k] = score.sentence_bleu(tok[j][i], tok[0][i], smoothing_function = score.SmoothingFunction().method3, weights=(1/2, 1/2, 0 , 0))
        k += 1

print('Kazakh to Russian:')
pd.DataFrame(data = kk_ru, columns = ['N', 'P', 'Z'])

Kazakh to Russian:


,N,P,Z
0,0.083333,0.066728,0.000000
1,0.094491,0.083388,0.094491
2,0.000000,0.000000,0.000000
3,0.094491,0.094491,0.094491
4,0.000000,0.000000,0.000000
5,0.057166,0.057166,0.057166
6,0.129099,0.109280,0.109280
7,0.048795,0.048795,0.048795
8,0.048795,0.048795,0.048795
9,0.066227,0.066227,0.066227


# Теперь оценки точности перевода посчитаны для трех пар языков и трех переводчиков. Проведем статистическое сравнение: 

Т.к в целом нет никаких предпосылок к знанию исходного распределения выборок, попробуем воспользоваться критерием знаков для двух связанных выборок (они, очевидно, связаны, т.к являются оценками точностей переводов одних предложений)

In [27]:
from statsmodels.stats.descriptivestats import sign_test # sign-test

In [28]:
p = []

for i in range(3):
    for j in range(i+1, 3):
        p.append(sign_test(en_de[:, i] - en_de[:, j])[1])
        
print('pvalue for hyp that translators are equal in en-to-de = ', p)

pvalue for hyp that translators are equal in en-to-de =  [1.0, 1.0, 1.0]


In [29]:
p = []

for i in range(3):
    for j in range(i+1, 3):
        p.append(sign_test(en_ru[:, i] - en_ru[:, j])[1])
        
print('pvalue for hyp that translators are equal in en-to-ru = ', p)

pvalue for hyp that translators are equal in en-to-ru =  [1.0, 1.0, 0.625]


In [30]:
p = []

for i in range(3):
    for j in range(i+1, 3):
        p.append(sign_test(kk_ru[:, i] - kk_ru[:, j])[1])
        
print('pvalue for hyp that translators are equal in kk-to-ru = ', p)

pvalue for hyp that translators are equal in kk-to-ru =  [0.25, 0.5, 1.0]


Как мы видим, нельзя выделить особых лидеров среди переводчиков. Но все же некая неоднородность в переводах с английского на русский и с казахского на русский есть. Чтобы понять, как именно распределены точности перевода между переводчиками, следаем следующее:

Попробуем с помощью бутстрепа построить доверительный интервал для средней разницы в точности перевода.

In [31]:
from arch.bootstrap import IIDBootstrap
interv = []

for i in range(3):
    for j in range(i+1, 3):
        bs = IIDBootstrap(en_ru[:,i] - en_ru[:,j]).conf_int(np.mean, 100, method = 'bca')
        interv.append([round(bs[0][0], 6), round(bs[1][0], 6)])

print(interv)

[[-0.001443, 0.0], [-0.001819, 0.001621], [-0.001287, 0.002309]]


In [32]:
from arch.bootstrap import IIDBootstrap
interv = []

for i in range(3):
    for j in range(i+1, 3):
        bs = IIDBootstrap(kk_ru[:,i] - kk_ru[:,j]).conf_int(np.mean, 100, method = 'bca')
        interv.append([round(bs[0][0], 6), round(bs[1][0], 6)])

print(interv)

[[0.0, 0.008646], [0.0, 0.022835], [-0.003331, 0.013346]]


По сгенерированным выборкам получаем следующий порядок точности перевода:

$en$_$ru$:  $Acc_p > Acc_z \cong Acc_n$

$kk$_$ru$: $Acc_n \ge Acc_p > Acc_z$

В целом же, пока что не удалось подобрать гиперпараметры и метрику из $nltk.translate.bleu$_$score$, чтобы отделить данные переводчики друг от друга более явно. Поэтому стоит весьма осторожно относиться к данным выше оценкам.